In [6]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

import traitlets
from tkinter import Tk, filedialog

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd 

In [7]:
class SelectFilesButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self):
        super(SelectFilesButton, self).__init__()
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Select Files"
        self.icon = "square-o"
        self.style.button_color = "orange"
        # Set on click behavior.
        self.on_click(self.select_files)

    @staticmethod
    def select_files(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        # List of selected fileswill be set to b.value
        b.files = filedialog.askopenfilename(multiple=True)

        b.description = "Files Selected"
        b.icon = "check-square-o"
        b.style.button_color = "lightgreen"

In [8]:
class DataContainer:
    def __init__(self, filePath):
        self.fileName = filePath
        self.readData()
        
    def readData(self):
        self.data = pd.read_csv(self.fileName, sep=',') 
        
        i=0
        names = []
        while i < len(self.data.columns):
            if i%2 == 1:
                names.append('Unnamed: '+str(i))
            i+=1

        names.append('Unnamed: '+str(len(self.data.columns)-1))  

        for k in names:
            self.data = self.data.drop([k], axis=1)

        self.data = self.data.drop(['S-Polarization Baseline', 'P-Polarization Baseline','Zero Correction'], axis=1)
        self.data = self.data.dropna()

        self.data = self.data.drop(self.data.index[0])
        self.wavelenghts = self.data.index

    def select_data(self,wavelenght,pol):
        ang = self.data.loc[wavelenght].index
        self.iza = []
        self.vza = []
        self.amp = []
        i = 0
        for k in ang:
            split = k.split('/')
            if split[0][0] == pol:
                self.vza.append(int(split[1]))
                self.iza.append(int(split[0][1:]))
                self.amp.append(float(self.data.loc[wavelenght][i]))
            i+=1
        return (self.iza,self.vza,self.amp)

In [9]:
def func(Wavelenght,Polarization):
    [x,y,z] = d1.select_data(Wavelenght, Polarization)  
  
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(x, y, z)
    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    ax.set_zlabel('Z Label')
    plt.show()
    return (x,y,z)
        

In [15]:
file_button = SelectFilesButton()
file_button


SelectFilesButton(description='Select Files', icon='square-o', style=ButtonStyle(button_color='orange'))

In [35]:
d1 = DataContainer(file_button.files[0])
k=interactive(func, Wavelenght=d1.wavelenghts,Polarization=widgets.RadioButtons(options=['s','p']))
display(k)

interactive(children=(Dropdown(description='Wavelenght', options=(' 780', ' 770', ' 760', ' 750', ' 740', ' 73…